## Setup

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('..')
import stober_synthesis_utils as stober

In [9]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2025_01_09_CompareOpt_80nmTarget_round3.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [10]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [11]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [55]:
sample_table_rmse = pd.read_csv('Sample_table_2025_01_09_CompareOpt_80nmTarget_Round3_rmsedist.csv')
sample_table_apdist = pd.read_csv('Sample_table_2025_01_09_CompareOpt_80nmTarget_Round3_rmsedist.csv')

In [56]:
stober.count_stock_vials(sample_table_rmse, 15000)

{'teos_count': 1, 'ammonia_count': 1, 'water_count': 1, 'ethanol_count': 1}

In [57]:
sample_table_apdist['ethanol_volume'].sum()/10000

1.335128747448325

## Load Tools

In [13]:
syringe_10 = syringe.HTTPSyringe.from_config(1, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_syringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_syringe.json")

In [14]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [15]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

## Load water syringe with water

In [16]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [17]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [18]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 10)

In [19]:
water_syringe.set_pulsewidth(water_syringe.full_position+150, s = 500)

In [20]:
water_syringe.load_syringe(740, water_syringe.full_position+150)

Loaded syringe, remaining volume 740 uL


## Load ammonia syringe

In [21]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [22]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [23]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [24]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [25]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+350, s = 500)

In [26]:
ammonia_syringe.load_syringe(370, ammonia_syringe.full_position+350)

Loaded syringe, remaining volume 370 uL


## Load TEOS syringe

In [31]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [32]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [33]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 100)

In [34]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+200, s = 200)

In [35]:
teos_syringe.load_syringe(680, teos_syringe.full_position+200)

Loaded syringe, remaining volume 680 uL


## load ethanol syringe

In [36]:
#jubilee.park_tool()

In [37]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [38]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [39]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+1, s = 10)

In [40]:
#mix_syringe.empty_position

In [41]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+150, s = 500)

In [42]:
mix_syringe.load_syringe(7100, mix_syringe.full_position+150)

Loaded syringe, remaining volume 7100 uL


## Load labware


In [79]:
samples = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples.manual_offset([(17.8,  174.6), (131.2, 175.3), (131.6, 105.4)])

New manual offset applied to septavialrev1_44_holder_2000ul


In [95]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks.manual_offset([(30.6, 264.2), (116.2, 264.8),  (117.0, 208.6)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [50]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [51]:
teos_stocks = [stocks[0]]
ammonia_stocks = [stocks[1]]
water_stocks = [stocks[2]]
ethanol_stocks = [stocks[3], stocks[4], stocks[5], stocks[6], stocks[7], stocks[8]]
rinse_stocks = [stocks[9], stocks[10], stocks[11]]

## Check sample alignment

In [103]:
jubilee.pickup_tool(teos_syringe)
for well in stocks:
    jubilee.move_to(x = well.x, y = well.y)
    print(well.y)
    jubilee.move_to(z = well.top_+7)
    ans = input(f'Is the tip centered over well?')
    if ans == 'y':
        continue
    else:
        continue

264.2


Is the tip centered over well? 


264.50307061616627


Is the tip centered over well? 


264.80614123233255


...

Is the tip centered over well? 


208.6036180422081


Is the tip centered over well? 


208.90668865837435


Is the tip centered over well? 


In [104]:
jubilee.park_tool()


In [92]:
#jubilee.pickup_tool(teos_syringe)
for i, row in sample_table_rmse.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

174.6


Is the tip centered over sample A1? 


174.69635205793185


Is the tip centered over sample A2? 


174.7927041158637


...

Is the tip centered over sample A8? 


160.62001836895595


Is the tip centered over sample B1? 


160.81272248481966


Is the tip centered over sample B3? 


In [87]:
jubilee.park_tool()


In [116]:
sample_table_rmse

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,dilute_well,well
0,0,12331b14-877d-40ca-a58e-5c4f523f2dd5,23.715099,90.229787,8.500000,1577.555114,1.600096,3.754775,0.004704,0.0,1700.0,C7,A1
1,1,1e0839f5-6ffa-464e-a72f-2d3b08508978,17.000000,88.969528,8.500000,1585.530473,2.232143,2.691584,0.003375,0.0,1700.0,C8,A2
2,2,f7126148-d164-4928-98cb-6356c1276f4a,56.949286,47.748529,229.399745,1365.902440,0.666320,9.016693,0.010923,567.0,1133.0,D1,A3
3,3,b73246e3-75cb-42da-b984-178d540a962a,17.000000,47.356099,89.896875,1545.747027,2.232143,2.691584,0.003344,0.0,1700.0,D2,A4
4,4,a6ee61ad-366f-4492-b19b-718cc6ee536d,27.579477,79.819448,28.604884,1563.996191,1.375894,4.366616,0.005456,0.0,1700.0,D3,A5
5,5,0d81ff88-5694-43fe-9b5b-681fb6e06f5c,158.871084,170.888522,38.707624,1331.532771,0.238850,25.153816,0.030726,1294.0,406.0,D4,A6
6,6,c73d7705-4597-4fbc-a4c1-75f6e9766d2e,149.280120,84.802900,112.964594,1352.952386,0.254196,23.635293,0.028776,1268.0,432.0,D5,A7
7,7,3690661f-ff2f-4829-b1eb-c1c810c3c228,179.701427,159.680656,113.209529,1247.408389,0.211164,28.451852,0.034326,1341.0,359.0,D6,A8
8,8,2ce5d5d6-746b-480e-90b3-a311867b03d4,124.614909,37.557242,255.000010,1282.827840,0.304510,19.730088,0.023666,1182.0,518.0,D7,B1
9,9,f1b612d2-55d2-49ed-830b-64f4a73ba162,24.106727,52.650063,81.418244,1541.824966,1.574101,3.816781,0.004743,0.0,1700.0,D8,B3


# Experiment


In [105]:
location_lookup_apdist = {row['uuid']:samples[row['well']] for i, row in sample_table_apdist.iterrows()}
location_lookup_rmse = {row['uuid']:samples[row['well']] for i, row in sample_table_rmse.iterrows()}

In [106]:
sample_table_apdist['ethanol_volume'].sum()

13351.287474483252

In [107]:
dilution_table_apdist = sample_table_apdist.copy()
dilution_table_rmse = sample_table_rmse.copy()

In [108]:
dilution_table_apdist['well'] = dilution_table_apdist['dilute_well']
dilution_table_rmse['well'] = dilution_table_rmse['dilute_well']

In [109]:
dilution_table_rmse

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,dilute_well,well
0,0,12331b14-877d-40ca-a58e-5c4f523f2dd5,23.715099,90.229787,8.500000,1577.555114,1.600096,3.754775,0.004704,0.0,1700.0,C7,C7
1,1,1e0839f5-6ffa-464e-a72f-2d3b08508978,17.000000,88.969528,8.500000,1585.530473,2.232143,2.691584,0.003375,0.0,1700.0,C8,C8
2,2,f7126148-d164-4928-98cb-6356c1276f4a,56.949286,47.748529,229.399745,1365.902440,0.666320,9.016693,0.010923,567.0,1133.0,D1,D1
3,3,b73246e3-75cb-42da-b984-178d540a962a,17.000000,47.356099,89.896875,1545.747027,2.232143,2.691584,0.003344,0.0,1700.0,D2,D2
4,4,a6ee61ad-366f-4492-b19b-718cc6ee536d,27.579477,79.819448,28.604884,1563.996191,1.375894,4.366616,0.005456,0.0,1700.0,D3,D3
5,5,0d81ff88-5694-43fe-9b5b-681fb6e06f5c,158.871084,170.888522,38.707624,1331.532771,0.238850,25.153816,0.030726,1294.0,406.0,D4,D4
6,6,c73d7705-4597-4fbc-a4c1-75f6e9766d2e,149.280120,84.802900,112.964594,1352.952386,0.254196,23.635293,0.028776,1268.0,432.0,D5,D5
7,7,3690661f-ff2f-4829-b1eb-c1c810c3c228,179.701427,159.680656,113.209529,1247.408389,0.211164,28.451852,0.034326,1341.0,359.0,D6,D6
8,8,2ce5d5d6-746b-480e-90b3-a311867b03d4,124.614909,37.557242,255.000010,1282.827840,0.304510,19.730088,0.023666,1182.0,518.0,D7,D7
9,9,f1b612d2-55d2-49ed-830b-64f4a73ba162,24.106727,52.650063,81.418244,1541.824966,1.574101,3.816781,0.004743,0.0,1700.0,D8,D8


In [110]:
location_lookup_dilute_apdist = {}

for i, row in dilution_table_apdist.iterrows():
    uuid_val = row['uuid']
    well = row['well']
    if well is not np.nan:
        well_loc = samples[well]
    else:
        well_loc = None

    location_lookup_dilute_apdist[uuid_val] = well_loc

In [111]:
location_lookup_dilute_rmse = {}

for i, row in dilution_table_rmse.iterrows():
    uuid_val = row['uuid']
    well = row['well']
    if well is not np.nan:
        well_loc = samples[well]
    else:
        well_loc = None

    location_lookup_dilute_rmse[uuid_val] = well_loc

In [112]:
dilution_table_rmse['ethanol_dilute_vol'].sum()

5652.0

## add ethanol

In [114]:
location_lookup_apdist

{'d25a22b3-7b31-4a34-a6c0-92e45b8ac925': Well B4 form septavialrev1_44_holder_2000ul on slot 2,
 '57821091-6396-45f7-a85b-d24a45526848': Well B5 form septavialrev1_44_holder_2000ul on slot 2,
 '853403e1-f5af-4354-ad1d-cb807eab1ccb': Well B6 form septavialrev1_44_holder_2000ul on slot 2,
 '50c1cea9-a08d-4bc6-b5d4-303a9e32d41b': Well B8 form septavialrev1_44_holder_2000ul on slot 2,
 '1e274033-bf13-44fd-8b73-443755e82430': Well C1 form septavialrev1_44_holder_2000ul on slot 2,
 '8b8f760d-9a31-422e-b8c3-da08e19161c6': Well C2 form septavialrev1_44_holder_2000ul on slot 2,
 'fd3adf41-2f06-47b7-9e04-e87b0ea1a731': Well C3 form septavialrev1_44_holder_2000ul on slot 2,
 'a383ef01-ff05-4691-8bf7-01e397948833': Well C4 form septavialrev1_44_holder_2000ul on slot 2,
 '295922f0-fd79-48b4-82ba-fbd227b98e2f': Well C5 form septavialrev1_44_holder_2000ul on slot 2,
 '9a0ea011-8f4a-4c93-89f9-5ef20af6cb7e': Well C6 form septavialrev1_44_holder_2000ul on slot 2}

In [113]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'ethanol_volume', ethanol_stocks[:3], stocks_usable_volume=15000, dwell_time = 10)

KeyboardInterrupt: 

In [65]:

stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'ethanol_volume', ethanol_stocks[:3], stocks_usable_volume=15000, dwell_time = 10)

In [66]:
#dispense remaining volume from etOH mix syringe
jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

In [67]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

## Dispense water


In [68]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'water_volume', water_stocks, dwell_time = 7)

In [69]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'water_volume', water_stocks, dwell_time = 7)

## dispense Ammonia

In [70]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'ammonia_volume', ammonia_stocks)

In [71]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'ammonia_volume', ammonia_stocks)

## mix precursors

In [72]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup_apdist, rinse_stocks, 5)

In [73]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup_rmse, rinse_stocks, 5)

## Add TEOS and mix

In [74]:
teos_start_time = time.time()

In [75]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks))

In [76]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks))

### Dilution

In [77]:
hrs5 = 60*60*5
while (time.time() - teos_start_time) < hrs5:
    time.sleep(60)

#dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table_apdist, location_lookup_dilute_apdist, 'ethanol_dilute_vol', ethanol_stocks[4:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table_apdist.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup_apdist[uuid_val]
    dilute_well = location_lookup_dilute_apdist[uuid_val]
    if dilute_well is None:
        continue
    else:
        vol = row['sample_dilute_vol']
        
        mix_syringe.aspirate(vol, sample_well.bottom(+5))
        time.sleep(10)
        mix_syringe.dispense(vol, dilute_well.bottom(+5))
        time.sleep(10)
        logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')
    
        for stock in rinse_stocks:
            mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    
        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks}')

    

In [78]:
# dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table_rmse, location_lookup_dilute_rmse, 'ethanol_dilute_vol', ethanol_stocks[4:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table_rmse.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup_rmse[uuid_val]
    dilute_well = location_lookup_dilute_rmse[uuid_val]
    if dilute_well is None:
        continue
    else:
        vol = row['sample_dilute_vol']
        
        mix_syringe.aspirate(vol, sample_well.bottom(+5))
        time.sleep(10)
        mix_syringe.dispense(vol, dilute_well.bottom(+5))
        time.sleep(10)
        logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')
    
        for stock in rinse_stocks:
            mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    
        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks}')

In [79]:
jubilee.park_tool()

In [82]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1)

In [83]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1)

In [84]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1)

In [80]:
sample_table_apdist

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,c3856eec-ecfa-46fe-8b51-b01796cff015,110.139117,71.798528,8.500000,1509.562356,0.344532,17.438158,0.021680,1114.0,586.0,A1,C7
1,1,65294301-7080-4705-b780-9d956b850445,79.743724,8.500000,106.194523,1505.561753,0.475855,12.625702,0.015594,891.0,809.0,A2,C8
2,2,b94af6ff-5138-4d36-a38f-991dd1dbae34,63.362727,73.475213,8.500000,1554.662060,0.598876,10.032124,0.012533,682.0,1018.0,A3,D1
3,3,92b3b03b-abef-4bad-9bcb-f7c67de18ccc,69.121179,54.357097,196.216759,1380.304965,0.548984,10.943851,0.013300,767.0,933.0,A4,D2
4,4,33dcd664-3e31-4fc5-8bbc-4e5b47b8914a,255.000010,18.585585,135.699256,1290.715149,0.148810,40.373762,0.048714,1447.0,253.0,A5,D3
5,5,0c1ddab6-f53a-45a3-8e2a-9f74b350eecd,17.000000,79.668913,255.000010,1348.331078,2.232143,2.691584,0.003253,0.0,1700.0,A6,D4
6,6,8e73357b-6307-4f73-ae08-8f0d3dbb60ef,17.000000,159.030688,255.000010,1268.969302,2.232143,2.691584,0.003233,0.0,1700.0,A7,D5
7,7,2958a316-ee3e-4f2c-83f0-9c8a85758448,76.725977,125.432878,255.000010,1242.841135,0.494571,12.147907,0.014542,859.0,841.0,A8,D6
8,8,7653c1b6-fc2e-468b-aeec-f130375c8f72,17.000000,186.999999,255.000010,1240.999991,2.232143,2.691584,0.003226,0.0,1700.0,B1,D7
9,9,310a128f-573a-4c59-a3b9-35ececd26c41,255.000010,58.881868,8.500000,1377.618122,0.148810,40.373762,0.049492,1447.0,253.0,B3,D8


In [81]:
sample_table_rmse

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,4aea3a58-a85b-42e6-bc69-aa5692cd7675,172.931279,8.500000,89.141196,1429.427525,0.219431,27.379945,0.033576,1327.0,373.0,A1,C7
1,1,03662924-3f8e-42e5-bd45-75fe757a3692,131.076677,8.755210,242.524165,1317.643948,0.289498,20.753170,0.024980,1208.0,492.0,A2,C8
2,2,bd1fe8eb-8834-4c41-9d4d-37c85d39a7d3,154.993822,151.000072,23.471911,1370.534195,0.244825,24.539935,0.030105,1284.0,416.0,A3,D1
3,3,2b52886d-aad5-4919-a877-25f98e29b245,243.006866,26.038005,162.100125,1268.855005,0.156154,38.474905,0.046271,1435.0,265.0,A4,D2
4,4,080f2aff-554d-434f-a78c-d6072afeabc2,245.898840,14.346618,128.395161,1311.359381,0.154317,38.932787,0.047086,1438.0,262.0,A5,D3
5,5,853457e3-042a-45a5-9dd9-4d50873e8801,32.411517,65.008919,255.000010,1347.579554,1.170770,5.131666,0.006200,0.0,1700.0,A6,D4
6,6,a42e6635-5215-4632-80a5-b635e6102482,17.000000,74.895387,8.500000,1599.604613,2.232143,2.691584,0.003379,0.0,1700.0,A7,D5
7,7,e4270ccf-97f1-4bad-821d-6c5019a2dfa0,69.863716,58.148577,71.332869,1500.654837,0.543149,11.061415,0.013694,777.0,923.0,A8,D6
8,8,fc10b4a6-f126-4443-bcd8-92514ee12e3b,76.588665,172.622596,184.226452,1266.562287,0.495458,12.126166,0.014615,858.0,842.0,B1,D7
9,9,40c911a0-8f77-481a-a569-d2d487717d2f,67.955414,144.173409,194.706745,1293.164431,0.558402,10.759277,0.012988,751.0,949.0,B3,D8


In [151]:
dilution_table_apdist

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,bbd1bfff-7563-4360-b5b8-86a1deb3928f,91.455773,121.416734,255.000010,1232.127483,0.414916,14.480053,0.017314,995.0,705.0,C7,C7
1,1,5b9ac046-6ca6-4ebc-87f3-e272edaedb7b,140.351853,8.500000,255.000010,1296.148137,0.270366,22.221694,0.026673,1240.0,460.0,C8,C8
2,2,9f782755-5ca4-4466-a1f1-a87ef665ed22,255.000010,42.990994,255.000010,1147.008985,0.148810,40.373762,0.047775,1447.0,253.0,D1,D1
3,3,f8b2207f-b63c-44d9-8fe4-70639fdd3f5c,255.000010,53.976205,8.500000,1382.523785,0.148810,40.373762,0.049511,1447.0,253.0,D2,D2
4,4,61431cc9-9557-45af-82c4-9e7acb251717,78.545307,77.696470,193.651897,1350.106326,0.483115,12.435958,0.015077,879.0,821.0,D3,D3
5,5,e773e91e-cdd8-466c-be37-8ff86cb7f610,52.806463,59.674126,39.147849,1548.371562,0.718594,8.360767,0.010419,478.0,1222.0,D4,D4
6,6,4559d494-fcc2-4dfc-a4cf-a14d93eb0029,102.107158,177.169546,8.500000,1412.223295,0.371633,16.166470,0.019945,1068.0,632.0,D5,D5
7,7,78e5ec30-0b59-4536-a9e0-21b4bac433a8,67.497684,128.016714,8.500000,1495.985602,0.562189,10.686805,0.013285,744.0,956.0,D6,D6
8,8,e42bc4d2-0157-425c-aed0-4cdec425d9ab,120.342861,146.735320,171.336467,1261.585352,0.315319,19.053701,0.022952,1164.0,536.0,D7,D7
9,9,4133ded3-a2c0-4d14-b4fb-578358436a04,17.000000,159.878042,8.500000,1514.621958,2.232143,2.691584,0.003355,0.0,1700.0,D8,D8
